In [1]:
import json
import time
import requests
from twilio.rest import Client
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# Load credentials.
external_dict = json.load(open("external_references.json"))

# Generate twilio client.
client = Client(external_dict["twilio_account_sid"], external_dict["twilio_auth_token"])

# Step One
driver = webdriver.Chrome()
driver.get(external_dict["step_one_url"])
username = driver.find_element(By.ID, "username")
username.send_keys(external_dict['username'])
password = driver.find_element(By.ID, "password")
password.send_keys(external_dict['password'])
step_one_button = driver.find_element(By.CLASS_NAME, external_dict["step_one_class_name"])
step_one_button.click()

# Step Two
step_two_button = driver.find_element(By.LINK_TEXT, external_dict["step_two_link_text"])
step_two_button.click()

# Wait for the text to hit and then obtain temporary passcode.
time.sleep(10)
passcode = int(client.messages.list()[0].body.split()[-1])

# Step Three.
driver.switch_to.window(driver.window_handles[-1])
step_three_input_box = (
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 
                                                                    external_dict["step_three_class_name"])))
)
step_three_input_box.send_keys(passcode)

# Forcing script to wait a couple seconds. Unknown input error otherwise.
time.sleep(2)
step_three_input_box.send_keys(Keys.ENTER)